# Prune

> Prune functions and classes

In [ ]:
#| default_exp prune

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.core import Module
from emb_opt.schemas import Item, Query, Batch, PruneFunction, PruneResponse

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export

class PruneModule(Module):
    def __init__(self,
                 function: PruneFunction,
                ):
        super().__init__(PruneResponse, function)
        
    def gather_inputs(self, batch: Batch) -> (List[Tuple], List[Query]):
        idxs, inputs = batch.flatten_queries()
        return (idxs, inputs)
    
    def scatter_results(self, batch: Batch, idxs: List[Tuple], results: List[PruneResponse]):
        for (q_idx, r_idx), result in zip(idxs, results):
            batch_item = batch.get_item(q_idx, r_idx)
            if result.data:
                batch_item.data.update(result.data)

            if not result.valid:
                batch_item.data['_internal']['remove'] = True
                batch_item.data['_internal']['remove_details'] = 'prune result invalid'

In [ ]:
batch = Batch(queries=[
                        Query(embedding=[0.1]),
                        Query(embedding=[0.2]),
                        Query(embedding=[0.3]),
                    ])

def prune_func(queries):
    return [PruneResponse(valid=i.embedding[0]>=0.2) for i in queries]

prune_module = PruneModule(prune_func)

batch = prune_module(batch)
assert [i.data['_internal'].get('remove', False) for i in batch] == [True, False, False]

In [ ]:
#| export

class TopKPrune():
    def __init__(self,
                 k: int,
                 agg: str='mean',
                 local: bool=True
                ):
        self.k = k
        self.agg = agg
        self.local = local
        assert agg in ['mean', 'max']
        
    def prune_queries(self, queries: List[Query]) -> List[PruneResponse]:
        scores = [np.array([j.score for j in i]) for i in queries]
        
        if self.agg=='mean':
            scores = np.array([i.mean() for i in scores])
        elif self.agg=='max':
            scores = np.array([i.max() for i in scores])
            
        topk_idxs = set(scores.argsort()[::-1][:self.k])
        
        outputs = [PruneResponse(valid=(i in topk_idxs), data={f'{self.agg}_score':scores[i]})
                  for i in range(len(queries))]
        
        return outputs
    
    def local_prune(self, queries: List[Query]) -> List[PruneResponse]:
        query_groups = defaultdict(list)
        idx_groups = defaultdict(list)
        
        outputs = [None for i in queries]
        
        for i, query in enumerate(queries):
            collection_index = query.data['_internal'].get('collection_index', -1)
            query_groups[collection_index].append(query)
            idx_groups[collection_index].append(i)
            
        for collection_idx, query_list in query_groups.items():
            prune_results = self.prune_queries(query_list)
            scatter_idxs = idx_groups[collection_idx]
            
            for i, result in enumerate(prune_results):
                outputs[scatter_idxs[i]] = result
                
        return outputs
    
    def __call__(self, queries: List[Query]) -> List[PruneResponse]:
        if self.local:
            outputs = self.local_prune(queries)
        else:
            outputs = self.prune_queries(queries)
            
        return outputs

In [ ]:
q1 = Query(embedding=[0.1])
q1.add_collection_index(0)
q1.add_query_results([
    Item(embedding=[0.11], score=-10),
    Item(embedding=[0.12], score=6),
])

q2 = Query(embedding=[0.2])
q2.add_collection_index(0)
q2.add_query_results([
    Item(embedding=[0.21], score=4),
    Item(embedding=[0.22], score=5),
])

q3 = Query(embedding=[0.3])
q3.add_collection_index(1)
q3.add_query_results([
    Item(embedding=[0.31], score=7),
    Item(embedding=[0.32], score=8),
])

queries = [q1, q2, q3]

prune_func = TopKPrune(k=1, agg='max', local=True)

assert [i.valid for i in prune_func(queries)] == [True, False, True]

prune_func = TopKPrune(k=1, agg='mean', local=True)

assert [i.valid for i in prune_func(queries)] == [False, True, True]

prune_func = TopKPrune(k=1, agg='mean', local=False)

assert [i.valid for i in prune_func(queries)] == [False, False, True]